### When we choose the model for our dataset,then it contains many parameter.So the process of adjusting the parameters to get the good performance is known as hyperparameter tuning<br><br>Here we will be learning about GridSearchCV method for Hyperparameter tuning

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [5]:
df = df[['Survived','Pclass','Sex','Fare']]

In [9]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Fare        0
dtype: int64

In [11]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()

In [12]:
df['sex'] = pd.DataFrame(lb.fit_transform(df['Sex']),columns=['sex'])

In [13]:
df.drop('Sex',axis = 1,inplace=True)

In [17]:
df

,Survived,Pclass,Fare,sex
0,0,3,7.2500,1
1,1,1,71.2833,0
2,1,3,7.9250,0
3,1,1,53.1000,0
4,0,3,8.0500,1
...,...,...,...,...
886,0,2,13.0000,1
887,1,1,30.0000,0
888,0,3,23.4500,0
889,1,1,30.0000,1


In [19]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=20)

In [21]:
from sklearn.model_selection import train_test_split
X = df.drop('Survived',axis=1)
Y = df['Survived']
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=43)

In [23]:
X_train.shape,X_test.shape,df.shape

((712, 3), (179, 3), (891, 4))

In [25]:
from sklearn.metrics import accuracy_score
rfc.fit(X_train,Y_train)
Y_pred = rfc.predict(X_test)
accuracy_score(Y_test,Y_pred)

0.7597765363128491

### Here a randomforest classifier gives 77.09 accuracy.Now lets improves the accuracy by performing hyperparameters tuning with GridSearchDV

In [28]:
# Number of trees in random forest
n_estimators = [20,60,100,120]

# Number of features to consider at every split
max_features = [0.2,0.6,1.0]

# Maximum number of levels in tree
max_depth = [2,8,None]

# Number of samples to train each tree
max_samples = [0.5,0.75,1.0]

# 108 diff random forest train

In [30]:
### Paramgrid contains the value of paramters of a model(which will be passed in GridSearchCV) which you want to hypertune.In my case there are 4 parameters of RandomForestClassifier model
param_grid = {
    'n_estimators': [50, 100, 150],          # Number of trees in the forest
    'max_features': ['auto', 'sqrt'],        # Number of features to consider at each split
    'max_depth': [None, 10, 20, 30],         # Maximum number of levels in tree
    'min_samples_split': [2, 5, 10],         # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],           # Minimum number of samples required at each leaf node
    'bootstrap': [True, False]    
}

In [32]:
from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier()
gsc = GridSearchCV(estimator=rf,
                   param_grid=param_grid,
                   verbose=2,
                   n_jobs=-1)

### In above,estimator means the model we want to find the paramters,param_grid contains the parameter of model we want to tune,n_jobs means how many core of cpu we want to utilize,n_jobs = -1 means we want to utilize all cores,verbose = 2 specifies the how detailly we want an output

In [ ]:
gsc.fit(X_train,Y_train)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


### Now we have fit our training dataset to gsc and we can find best parameters for our model as

In [95]:
BEST_model = gsc.best_params_ ## We make our own model with these best parameters

In [97]:
gsc.best_params_

{'bootstrap': False,
 'max_depth': 10,
 'max_features': 'sqrt',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 50}

### When we put those parameters ,then we will be getting a best accuracy.We can find a best accuracy as

In [100]:
gsc.best_score_

0.8300403821530582

In [102]:
Yp = gsc.predict(X_test)
accuracy_score(Y_test,Yp)

0.7821229050279329